# Codigo propio

In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-VALIDATION-10.csv'
VALIDATION_FILE = 'DATASET-TEST-10.csv'

TRAINING_BATCH_SIZE = 32
TRAINING_DATA_AUG = False
TRAINING_BALANCED = False
TRAINING_SHUFFLE = False
TRAINING_PREFETCH = None
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 32
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_SHUFFLE = False
VALIDATION_PREFETCH = None
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    shuffle=TRAINING_SHUFFLE,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                shuffle=VALIDATION_SHUFFLE, 
                                                size=VALIDATION_TAKE_SIZE)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5), strides=2, input_shape=IMAGE_SIZE),
    tf.keras.layers.ReLU(), # 270, 270
    tf.keras.layers.Conv2D(32,(7,7), strides=5),
    tf.keras.layers.ReLU(), # 54, 54
    tf.keras.layers.Conv2D(32,(3,3), strides=2),
    tf.keras.layers.ReLU(), # 27, 27
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(DR_LEVELS_PER_CLASS), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/EYEPACS_Seq_Modelo_propio 2', histogram_freq=1, write_graph=False)]

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
# See Tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2790), started 1 day, 0:39:08 ago. (Use '!kill 2790' to kill it.)

In [12]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset, verbose=1, callbacks=cbacks)

Epoch 1/50
222/222 [==============================] - 26s 118ms/step - loss: 0.5842 - accuracy: 0.7460 - val_loss: 0.5863 - val_accuracy: 0.7354
Epoch 2/50
222/222 [==============================] - 26s 116ms/step - loss: 0.5700 - accuracy: 0.7462 - val_loss: 0.5808 - val_accuracy: 0.7354
Epoch 3/50
222/222 [==============================] - 26s 116ms/step - loss: 0.5649 - accuracy: 0.7467 - val_loss: 0.5791 - val_accuracy: 0.7354
Epoch 4/50
222/222 [==============================] - 26s 116ms/step - loss: 0.5586 - accuracy: 0.7467 - val_loss: 0.5760 - val_accuracy: 0.7354
Epoch 5/50
222/222 [==============================] - 26s 115ms/step - loss: 0.5569 - accuracy: 0.7467 - val_loss: 0.5750 - val_accuracy: 0.7354
Epoch 6/50
222/222 [==============================] - 26s 116ms/step - loss: 0.5553 - accuracy: 0.7467 - val_loss: 0.5759 - val_accuracy: 0.7354
Epoch 7/50
222/222 [==============================] - 26s 116ms/step - loss: 0.5539 - accuracy: 0.7467 - val_loss: 0.5720 - val_ac